In [1]:
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

from datasets import load_dataset, load_metric, Dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

import numpy as np
from typing import Dict
import torch
import numpy as np

import ray
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
import ray.data
import ray.train
from ray.train.huggingface.transformers import prepare_trainer, RayTrainReportCallback
from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler
from tqdm import tqdm
from sklearn.metrics import f1_score


# Constants

In [2]:
train_path = "./data/train"
valid_path = "./data/dev"

num_labels = 3
metric_name = "accuracy"

model_checkpoint = "distilbert-base-uncased"
model_name = model_checkpoint.split("/")[-1]
validation_key = "validation"
task = "cola"

name = f"{model_name}-finetuned-{task}"
actual_task = task
batch_size = 16
num_workers = 1
use_gpu = True

tune_epochs = 4


# Read data

In [3]:
with open(train_path + ".input0", 'r') as f,  open(train_path + ".label", 'r') as g:
    train_texts = f.read().split("\n")[:-1]
    train_labels = g.read().split("\n")[:-1]
    
with open(valid_path + ".input0", 'r') as f,  open(valid_path + ".label", 'r') as g:
    valid_texts = f.read().split("\n")[:-1]
    valid_labels = g.read().split("\n")[:-1]

In [4]:
df_train = pd.DataFrame([train_texts, train_labels]).T
df_train.columns = ["text", 'label']
df_train['label'] = df_train['label'].astype(int)
train_dataset = Dataset.from_pandas(df_train)

In [5]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 29412
})

In [6]:
df_test = pd.DataFrame([valid_texts, valid_labels]).T
df_test.columns = ["text", 'label']
df_test['label'] = df_test['label'].astype(int)
test_dataset = Dataset.from_pandas(df_test)

In [7]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 7353
})

# Ray training

In [8]:
ray_datasets = {
    "train": ray.data.from_huggingface(train_dataset),
    "validation": ray.data.from_huggingface(test_dataset),
}
ray_datasets

2023-12-25 18:26:01,394	INFO worker.py:1724 -- Started a local Ray instance.


{'train': MaterializedDataset(
    num_blocks=1,
    num_rows=29412,
    schema={text: string, label: int64}
 ),
 'validation': MaterializedDataset(
    num_blocks=1,
    num_rows=7353,
    schema={text: string, label: int64}
 )}

In [9]:

# Tokenize input sentences
def collate_fn(examples: Dict[str, np.array]):
    outputs =  tokenizer(list(examples['text']), truncation=True,  
                         padding="longest",
                         return_tensors="pt",)

    outputs["labels"] = torch.LongTensor(examples["label"])

    # Move all input tensors to GPU
    for key, value in outputs.items():
        outputs[key] = value.cuda()
    return outputs

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Calculate the maximum steps per epoch based on the number of rows in the training dataset.
# Make sure to scale by the total number of training workers and the per device batch size.
max_steps_per_epoch = ray_datasets["train"].count() // (batch_size * num_workers)


def train_func(config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")

    metric = load_metric("glue", actual_task)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, num_labels=num_labels
    )

    train_ds = ray.train.get_dataset_shard("train")
    eval_ds = ray.train.get_dataset_shard("eval")

    train_ds_iterable = train_ds.iter_torch_batches(
        batch_size=batch_size, collate_fn=collate_fn
    )
    eval_ds_iterable = eval_ds.iter_torch_batches(
        batch_size=batch_size, collate_fn=collate_fn
    )

    print("max_steps_per_epoch: ", max_steps_per_epoch)

    args = TrainingArguments(
        name,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=config.get("learning_rate", 2e-5),
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        push_to_hub=False,
        max_steps=max_steps_per_epoch * config.get("epochs", 2),
        disable_tqdm=True,  # declutter the output a little
        no_cuda=not use_gpu,  # you need to explicitly set no_cuda if you want CPUs
        report_to="none",
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_ds_iterable,
        eval_dataset=eval_ds_iterable,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.add_callback(RayTrainReportCallback())

    trainer = prepare_trainer(trainer)

    print("Starting training")
    trainer.train()

In [11]:

trainer = TorchTrainer(
    train_func,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={
        "train": ray_datasets["train"],
        "eval": ray_datasets["validation"],
    },
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)

In [12]:
result = trainer.fit()

(TorchTrainer pid=92340) Started distributed worker processes: 
(TorchTrainer pid=92340) - (ip=192.168.0.2, pid=92417) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=92417) Setting up process group for: env:// [rank=0, world_size=1]


(RayTrainWorker pid=92417) Is CUDA available: True


(RayTrainWorker pid=92417) /tmp/ipykernel_91028/683511016.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
(SplitCoordinator pid=92472) Auto configuring locality_with_output=['1a66715af743dabd9b80b3a893f9972e0380dae976dd72512d19bc2a']
(RayTrainWorker pid=92417) /home/itmo/miniconda3/envs/automl/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/glue/glue.py
(RayTrainWorker pid=92417) You can avoid this message in future by passing the argument `trust_remote_code=True`.
(RayTrainWorker pid=92417) Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
(Ray

(RayTrainWorker pid=92417) max_steps_per_epoch:  1838


(RayTrainWorker pid=92417) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
(RayTrainWorker pid=92417) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(RayTrainWorker pid=92417) Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

(pid=92473) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=92417) {'eval_loss': 0.7379251718521118, 'eval_matthews_correlation': 0.4091076801455507, 'eval_runtime': 13.4609, 'eval_samples_per_second': 546.25, 'eval_steps_per_second': 34.173, 'epoch': 1.5}


(RayTrainWorker pid=92417) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/itmo/ray_results/TorchTrainer_2023-12-25_18-26-05/TorchTrainer_10d8a_00000_0_2023-12-25_18-26-05/checkpoint_000001)


(RayTrainWorker pid=92417) {'train_runtime': 352.7186, 'train_samples_per_second': 166.75, 'train_steps_per_second': 10.422, 'train_loss': 0.7170052844889145, 'epoch': 1.5}


2023-12-25 18:32:09,928	INFO tune.py:1042 -- Total run time: 364.40 seconds (364.37 seconds for the tuning loop).


# Tuning

In [13]:
tuner = Tuner(
    trainer,
    param_space={
        "train_loop_config": {
            "learning_rate": tune.grid_search([2e-5, 2e-4, 2e-3, 2e-2]),
            "epochs": tune_epochs,
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=1,
        scheduler=ASHAScheduler(
            max_t=tune_epochs,
        ),
    ),
    run_config=RunConfig(
        name="tune_transformers",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)

2023-12-25 18:32:09,938	INFO tuner_internal.py:401 -- A `RunConfig` was passed to both the `Tuner` and the `TorchTrainer`. The run config passed to the `Tuner` is the one that will be used.


In [14]:
tune_results = tuner.fit()


(TorchTrainer pid=97812) Started distributed worker processes: 
(TorchTrainer pid=97812) - (ip=192.168.0.2, pid=97865) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=97865) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=97865) /tmp/ipykernel_91028/683511016.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
(SplitCoordinator pid=97919) Auto configuring locality_with_output=['1a66715af743dabd9b80b3a893f9972e0380dae976dd72512d19bc2a']


(RayTrainWorker pid=97865) Is CUDA available: True


(RayTrainWorker pid=97865) /home/itmo/miniconda3/envs/automl/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/glue/glue.py
(RayTrainWorker pid=97865) You can avoid this message in future by passing the argument `trust_remote_code=True`.
(RayTrainWorker pid=97865) Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
(RayTrainWorker pid=97865)   warnings.warn(
(RayTrainWorker pid=97865) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
(RayTrainWorker pid=97865) You should probably TRAIN this model on a down-stream ta

(RayTrainWorker pid=97865) max_steps_per_epoch:  1838
(RayTrainWorker pid=97865) Starting training


(RayTrainWorker pid=97865) /tmp/ipykernel_91028/1795508781.py:7: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)


...

(RayTrainWorker pid=114330) {'loss': 1.1552, 'learning_rate': 0.01499727965179543, 'epoch': 0.25}


(SplitCoordinator pid=114384) Auto configuring locality_with_output=['1a66715af743dabd9b80b3a893f9972e0380dae976dd72512d19bc2a']
(SplitCoordinator pid=114385) Executing DAG InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=114385) Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=1.0, gpu=1.0, object_store_memory=0.0), locality_with_output=['1a66715af743dabd9b80b3a893f9972e0380dae976dd72512d19bc2a'], preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(SplitCoordinator pid=114385) Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


(pid=114385) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=114330) {'eval_loss': 0.9774772524833679, 'eval_matthews_correlation': 0.0, 'eval_runtime': 13.1348, 'eval_samples_per_second': 559.81, 'eval_steps_per_second': 35.021, 'epoch': 0.25}


2023-12-25 18:53:23,875	INFO tune.py:1042 -- Total run time: 1273.91 seconds (1273.89 seconds for the tuning loop).
(RayTrainWorker pid=114330) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/itmo/ray_results/tune_transformers/TorchTrainer_ea11d_00003_3_learning_rate=0.0200_2023-12-25_18-32-09/checkpoint_000000)


In [99]:
path = tune_results[0].checkpoint.path + "/checkpoint"
model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=3).cuda()

In [100]:
y_pred = []
y_true = []
for example in tqdm(test_dataset):
    with torch.no_grad():
        inputs = tokenizer(example['text'], return_tensors="pt")
        logits = model(input_ids=inputs['input_ids'].cuda()[:,:512], 
                      attention_mask= inputs['attention_mask'].cuda()[:,:512]).logits
        prediction = logits.argmax().item()
        y_true.append(example['label'])
        y_pred.append(prediction)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7353/7353 [00:34<00:00, 212.56it/s]


In [101]:
score = f1_score(y_pred, y_true, average=None)
score

array([0.72418478, 0.65909677, 0.36818409])

In [102]:
score.mean()

0.5838218829494224